# Slack Integration For Cloudability

Get instant notifcation of unusual cloud spend and eliminate surprises by integrating Cloudability anomalies with your slack using this script. 

Steps:

1. Enter your slack webhook below, for information on how to create webhooks see here: https://api.slack.com/messaging/webhooks
2. Enter your Cloudability API Token
3. Optional: Choose how far you want lookback
4. Optional: Schedule this script to run once a week/day




In [ ]:
###Enter details below
Hook = "https://hooks.slack.com/services/example"

token = "example token"

#Anomalies for the last x days
Lookback_Days = 7

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import requests 
import base64
import json
import time
from pandas.io.json import json_normalize

########--Variables--########

#encode token for html
token2 = token+":"
message_bytes = token2.encode('ascii')
base64_bytes = base64.b64encode(message_bytes)
base64_message = base64_bytes.decode('ascii')
#API details
headers = {
  'Content-Type': 'application/json',
  'Authorization': 'Basic '+base64_message
}

headers_slack = {
  'Content-Type': 'application/json'
}

########--Functions--########

#query cloudability, returns dataframe (table)
def Query_Cloudability(url, headers, payload):
    response = requests.request("GET", url, headers=headers, data = payload)
    print("response code from Cloudability data pull: "+str(response.status_code))
    if "v3" in url:
        df = json_normalize(response.json(), record_path=['result'])
    else:
        df = json_normalize(response.json())
    return df

########--Code--########

#Get Cloudability anomalie data for defined period (usually last 7 days)
Today = time.strftime('%Y-%m-%d', time.gmtime())
Start_Date = time.strftime('%Y-%m-%d', time.gmtime(time.mktime(time.gmtime()) - (Lookback_Days*86400)))
url = 'https://api.cloudability.com/v3/anomalies?endDate='+Today+'&startDate='+Start_Date+''
df = Query_Cloudability(url, headers, '')

#Clean data
df2 = df[["date","enhancedServiceName","usageFamily","unusualSpend","vendorAccountName"]]
df2['unusualSpend'] = df2['unusualSpend'].apply(pd.to_numeric).round(1)
df2.columns = ['*Date: *', '*Service Name: *', '*Usage Family: *', '*Unusual Spend: *', '*Account Name: *' ]
df2 = df2.sort_values(by=['*Unusual Spend: *'],ascending=False)

#Create and post slack message
Payload_Intro = "{\"text\":\"*Anomaly Alert!*\\n<!channel>\\nFor more detail click here: https://app.cloudability.com/#/anomalies\\n_____________________________________________________________________\"}"
response = requests.request("POST", Hook , headers=headers_slack, data = Payload_Intro)
print("response code for POST to slack: "+str(response.status_code))

for ind in df2.head(5).index:
    Date = df2.loc[ind,["*Date: *"]].to_string()
    Service_name = df2.loc[ind,["*Service Name: *"]].to_string()
    usageFamily = df2.loc[ind,["*Usage Family: *"]].to_string()
    unusualSpend = df2.loc[ind,["*Unusual Spend: *"]].to_string()
    vendorAccountName = df2.loc[ind,["*Account Name: *"]].to_string()
    payload3 = "{\"text\":\"" + Date + "\\n\\n" + Service_name + "\\n" + usageFamily + "\\n" + unusualSpend + "\\n" + vendorAccountName + "\\n\\n---------------------------------------------------\"}"
    response3  = requests.request("POST", Hook , headers=headers_slack, data = payload3)
    print("response code for POST to slack: "+str(response3.status_code))

